### TOPUP 

1. acceder al arbol de directorio
2. recuperar los datos que son importantes: echotime totalreadtime (acqparameters.txt)
3. coger solo la primera imagen B0
4. brain mask
5. subprocess

In [1]:
import os
import nipype
import json
from pathlib import Path

200219-15:28:11,745 nipype.utils INFO:
	 No new version available.


In [3]:
#path="C:\Users\viole\Desktop\TFG\SENECA-PICASO"

In [4]:
#camino = Path(main+'Documentos/archivos_texto/')

In [5]:
sep=os.path.sep
main= sep+'Volumes'+sep+'HDD1'+sep+'INVESTIGACION'+sep+'SENECA-PICASSO'

In [6]:
rootDir=main+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'func'+sep+'sub-sp01'+sep+'ses-02_task-rest_bold.json'

for dirName, subdirList, fileList in os.walk(rootDir, topdown=False):
    print('hola')
    print('Found directory: %s' % fileList)


In [7]:

os.access("nifti", os.R_OK) 

False

1. Read the json files 

In [60]:
file=main+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'fmap'+sep+'sub-sp01ses-02_acq-dwi_dir-1_epi.json'

In [61]:
with open(file) as json_file:
    data = json.load(json_file)
    EES=data["EffectiveEchoSpacing"]
    AdqMatPE=data["AcquisitionMatrixPE"]
    PhaseDir=data["PhaseEncodingDirection"]
    print(EES, AdqMatPE, PhaseDir)


0.0007 128 j-


2. Calculate TotalReadOutTime

In [62]:
ReadOut= 0.5*EES*(AdqMatPE-1)

In [63]:
ReadOut

0.044449999999999996

3. Creat acqparameters.txt

In [69]:
string=PhaseDir
f= open(os.path.join(main,"acqparameters.txt"),"w+")
f.write(string.replace('j-', '0 -1 0 ').replace('j','0 1 0 ')+str(ReadOut)) 
f.close()

4. Make comandos 

In [13]:
#fslroi sub-sp01ses-02_dir-AP_dwi.nii nodif 0 1
#sub-sp01ses-02_dir-AP_dwi.nii

In [30]:
from nipype.interfaces.fsl import ExtractROI
dwidata=main+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'dwi'+sep+'sub-sp01ses-02_dir-AP_dwi.nii'
root=main+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'dwi'+sep
fslroi = ExtractROI(in_file=dwidata, roi_file=main+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'dwi'+sep+'nodif.nii.gz', t_min=0,t_size=1)
fslroi.cmdline #== 'fslroi %s nodif.nii 0 1' % dwidata
fslroi.run()

In [31]:
from nipype.interfaces.fsl import ExtractROI
dwidata=main+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'dwi'+sep+'sub-sp01ses-02_dir-AP_dwi.nii'
root=main+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'dwi'+sep
fslroi = ExtractROI(in_file=dwidata, roi_file=main+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'func'+sep+'nodif_PA.nii.gz',t_min=0,t_size=1)
fslroi.cmdline #== 'fslroi %s nodif.nii 0 1' % dwidata
fslroi.run()

In [21]:
#fls merge  -t AP_PA_b0 nodif nodif_PA
#C:\Users\viole\Desktop\TFG\SENECA-PICASO\nifti\nifti\sub-sp01\ses-02

In [49]:
root=main+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'
import subprocess

In [33]:
from nipype.interfaces.fsl import Merge
merger = Merge()
merger.inputs.in_files = [root+sep+'dwi'+sep+'nodif.nii.gz', root+sep+'func'+sep+'nodif_PA.nii.gz']
merger.inputs.dimension = 't'
merger.inputs.output_type = 'NIFTI_GZ'
merger.cmdline
#'fslmerge -t functional2_merged.nii.gz functional2.nii functional3.nii'
#merger.inputs.tr = 2.25
#merger.cmdline
#'fslmerge -tr functional2_merged.nii.gz functional2.nii functional3.nii 2.25'
res=merger.run()

In [34]:
from nipype.interfaces.fsl import Info
Info.version()
Info.output_type()

'NIFTI_GZ'

In [35]:
#topup --imain=AP_PA_b0 --datain=acqparams.txt --config=b02b0.cnf --out=topup_AP_PA_b0
#imain=AP_PA_b0.nii.gz
#datain=acqparams.txt
#config=b02b0.cnf(?)
#out=topup_AP_PA_b0

In [51]:
main

'/Volumes/HDD1/INVESTIGACION/SENECA-PICASSO'

In [57]:
from nipype.interfaces.fsl import TOPUP
imain=main+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'nodif_merged.nii.gz'
params=main+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'acqparams.txt'
topup = TOPUP()
topup.inputs.in_file = imain
topup.inputs.encoding_file = params
topup.inputs.output_type = "NIFTI_GZ"
topup.cmdline 
#'topup --config=b02b0.cnf --datain=topup_encoding.txt --imain=b0_b0rev.nii --out=b0_b0rev_base --iout=b0_b0rev_corrected.nii.gz --fout=b0_b0rev_field.nii.gz --logout=b0_b0rev_topup.log'
res = topup.run() 

----

In [ ]:
from nipype.interfaces.fsl import ApplyTOPUP
applytopup = ApplyTOPUP()
imain_up  = main+sep+'nifti'+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'dwi'+sep+'sub-sp01ses-02_dir-AP_dwi.nii'
imain_down= main+sep+'nifti'+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'func'+sep+'ses-02_task-rest_bold.nii'
params='Users'+sep+'administrator'+sep+'Downloads'
applytopup.inputs.in_files = [imain_up,imain_down]
applytopup.inputs.encoding_file = params+sep+'topup_encoding.txt'
applytopup.inputs.in_topup_fieldcoef = params+sep+"topup_fieldcoef.nii.gz"#de donde ha salido
applytopup.inputs.in_topup_movpar = params+sep+"topup_movpar.txt"#de donde ha salido
applytopup.inputs.output_type = "NIFTI_GZ"
applytopup.cmdline 
res = applytopup.run() 